<a href="https://colab.research.google.com/github/okura1406/radnlp-2024/blob/main/radnlp2024_gemini_2_0_Meta_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#　説明

gemini-2.0でプロンプト修正

修正後の誤分類をGPTo1で再度修正

 TNM分類の単位を㎝→㎜に変更



# データの用意

まずは Colab 環境で扱えるようにデータを用意するところから始めます。データは Google Drive に用意して、これをマウントすることでアクセスできるようにします。

In [ ]:
# Google Drive をマウントするためのコード.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# data directory の設定. 自分の保存先に合わせて以下の path は変更が必要です.
data_dir = "/content/drive/MyDrive/"
valid_df = pd.read_csv(os.path.join(data_dir, "label.csv"))

# valid_df の中身を確認.
display(valid_df.head(3))

,id,t,n,m
0,147290,T2a,N0,M0
1,241752,Tis,N0,M0
2,876951,T2a,N0,M0


valid_df にはテキストの id と TNM分類の正解ラベルが格納されているのが確認できました.

# モデルの用意



In [ ]:
# GPU の確認.
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


今回は L4 GPU を用いて推論を行なっていきます。

In [ ]:
# パッケージのインストール
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

# 環境変数の準備（左端の鍵アイコンでGEMINI_API_KEYを設定）
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

# Markdown出力ヘルパーの準備
def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

    # モデル一覧の表示
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
mod

In [ ]:
# モデルの準備
model = genai.GenerativeModel("models/gemini-2.0-flash-thinking-exp-1219")


# 推論

では、今回の RadNLP のデータで本番の推論に移ります。


In [ ]:
# ==============================================
# 1) 必要パッケージのインストール / インポート
# ==============================================
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

import os
import re
import json
import pandas as pd
from IPython.display import display

# ==============================================
# 2) Google Generative AI の設定
#    (事前にColabの鍵アイコンで GEMINI_API_KEY を登録しておく)
# ==============================================
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))


# レポート本文を取得する関数
def get_report(report_id):
    report_path = os.path.join(data_dir, f"{report_id}.txt")
    with open(report_path, "r", encoding="utf-8") as f:
        report = f.read()
    return report

# ==============================================
# 3) JSON抽出用の関数
# ==============================================
def extract_json(text):
    """
    出力された文章からJSON部分を抜き出し、Python辞書に変換する関数。
    失敗したらデモデータを返す。
    """
    json_pattern = r'```json\n(.*?)\n```|(\{.*\})'
    matches = re.findall(json_pattern, text, re.DOTALL)

    parsed_json = {}
    for match in matches:
        json_candidate = match[0] if match[0] else match[1]
        try:
            parsed_json = json.loads(json_candidate)
            break  # 見つかった時点で終了
        except json.JSONDecodeError:
            # 失敗した場合はデフォルトの値を返す
            parsed_json = {
                "T": {
                    "stage": "T2b",
                    "details": "37mmの腫瘤影が認められる"
                },
                "N": {
                    "stage": "N0",
                    "details": "縦隔に有意なリンパ節腫大は認められません"
                },
                "M": {
                    "stage": "M0",
                    "details": "胸水はありません。"
                }
            }
    return parsed_json

# ==============================================
# 4) 推論に使用するプロンプトと関数
# ==============================================
prompt = """「以下の文書は私自身が保有するデータであり、著作権に関する問題はありません。
  【instruction】を参考に肺癌について記載された読影レポートをTNM分類で判定し指定の形式にまとめてください。」

【instruction】
{
  "TNM分類_第8版_2017年": {
    "T_原発腫瘍": {
      "TX": "原発腫瘍の存在が判定できない、または喀痰・気管支洗浄液細胞診でのみ陽性で画像・気管支鏡では観察不能",
      "T0": "原発腫瘍を認めない",
      "Tis": "上皮内癌(carcinoma in situ)：肺野型の場合、充実成分径0mmかつ病変全体径≦30mm",
      "T1": {
        "定義": "腫瘍の充実成分径≦30mm、肺または臓側胸膜に覆われ、主気管支への浸潤なし",
        "T1mi": "微小浸潤性腺癌：部分充実型、充実成分径≦5mm、病変全体径≦30mm",
        "T1a": "充実成分径≦10mm、Tis・T1mi以外",
        "T1b": "充実成分径＞10mmかつ≦20mm",
        "T1c": "充実成分径＞20mmかつ≦30mm"
      },
      "T2": {
        "定義": "充実成分径＞30mmかつ≦50mm、または充実成分径≦30mmでも以下の条件を満たす場合:\n1) 主気管支浸潤(気管分岐部は除く)\n2) 臓側胸膜浸潤\n3) 肺門まで連続する無気肺または閉塞性肺炎",
        "T2a": "充実成分径＞30mmかつ≦40mm",
        "T2b": "充実成分径＞40mmかつ≦50mm"
      },
      "T3": {
        "定義": "充実成分径＞50mmかつ≦70mm、または充実成分径≦50mmでも以下のいずれか:\n1) 壁側胸膜、胸壁(上部肺溝腫瘍含む)、横隔神経、心膜への直接浸潤\n2) 同一葉内の不連続な副腫瘍結節"
      },
      "T4": {
        "定義": "充実成分径＞70mm、または以下いずれか:\n1) 横隔膜、縦隔、心臓、大血管、気管、反回神経、食道、椎体、気管分岐部への浸潤\n2) 同側の異なる肺葉内に不連続な副腫瘍結節"
      }
    },
    "N_所属リンパ節": {
      "NX": "所属リンパ節評価不能",
      "N0": "所属リンパ節転移なし",
      "N1": "同側の気管支周囲、肺門、肺内リンパ節転移（原発腫瘍への直接浸潤含む）",
      "N2": "同側縦隔、気管分岐下リンパ節転移",
      "N3": "対側縦隔、対側肺門、前斜角筋・鎖骨上窩リンパ節転移"
    },
    "M_遠隔転移": {
      "M0": "遠隔転移なし",
      "M1": {
        "定義": "遠隔転移あり",
        "M1a": "対側肺内副腫瘍結節、胸膜・心膜結節、悪性胸水・悪性心嚢水（同側・対側問わず）",
        "M1b": "肺以外の一臓器への単発遠隔転移",
        "M1c": "肺以外の一臓器または多臓器への多発遠隔転移"
      }
    }
  },

【T分類判定のステップ】
*腫瘍の有無の確認（最重要）
読影レポートを読み込み、最初に「腫瘍を認めない」、「悪性所見なし」などの明示的な記載の有無を厳格にチェック。
腫瘍を認めない場合 ⇒ T0。
原発腫瘍の存在が判定できない or 細胞診のみ陽性(画像で捉えられない) ⇒ TX。

*大まかなT分類の判定
レポートに “胸壁浸潤” “多発結節” など明らかなキーワードがあれば該当のT(T3/T4)に直行。
なければ 病変の充実成分径、GGO性状、主気管支浸潤の有無などを精査。
胸壁・大血管浸潤など Tを上げる決定的所見(T3/T4 所見)がないか優先チェックし、なければサイズ基準で T1 / T2 を振り分ける。
T1 or T2 と判定した場合、充実成分径の大きさから T1a, T1b, T1c, T2a, T2b に細かく振り分ける。
混同しやすい境界(30mm, 40mm, 50mm) は特に注意。
例: 30mm 付近 ⇒ T1c と T2a の両方を再評価し、他の情報と合わせて最終決定。

*浸潤部位の確認
臓側胸膜 vs 壁側胸膜の記載ゆれにも注意し、壁側胸膜への明確な浸潤があればT3(さらに大きさ次第でT4)とする。
“大血管”“心臓”“椎体”などのキーワード ⇒ T4 を強く疑う。

【Tステージ誤り防止のための重要事項】
1. 早期がん（Tis, T1mi, T0）の過大評価に注意！
充実成分径が非常に小さい(≦5mm)場合は、安易に T1a〜T1c とせず T1mi や Tis の可能性を優先検討。
Tis: 充実成分径0 mm、かつ病変全体30 mm以下。(充実成分径がわずかでも認められる場合はTisの可能性を排除)
T1mi: 部分充実 (mixed GGO) で、solid 部分≦5 mm、全体≦30 mm。(充実成分径が5mmを超えている場合はT1a以降を検討)
画像所見で浸潤が明らかでない場合 ⇒ まず早期がんの可能性を強く意識し、過大評価を避ける。

2. 進行がん（T3, T4）の過小評価に注意！　大径腫瘍・浸潤所見を見落とさない
腫瘍が50mmを超える場合 ⇒ T3 以上を強く検討。
70mm超や胸壁・大血管・心臓などへの浸潤が記載されていればT4の可能性大。
壁側胸膜浸潤、胸壁侵犯、横隔膜・大血管・気管分岐部浸潤などのキーワードを見落とさず、T4を積極的に検討する。

 【判定において特に注意すべき点】
"GGO関連":
純粋GGO(pure GGO)か部分的solidを含むmixed GGOかを明確に抽出し、充実成分の径を厳密に評価。
pure GGO (充実成分0mm)で病変全体30mm以下なら Tis をまず検討。
mixed GGO で 充実成分≦5mm かつ病変全体≦30mm なら T1mi を最優先で検討。

"副腫瘍結節":
同一葉内ならT3、同側の別葉ならT4、対側ならM1a等、位置関係を正しくチェック。
“不連続な結節”か単なる“直接浸潤”かの区別にも留意する。

"主要気管支・縦隔浸潤":
気管分岐部まで及ぶか、胸壁・大血管・横隔膜・心臓などへの直接浸潤があれば、T4を優先的に考慮。

"既知病変と新規原発巣":
レポートで「既知の増悪」「再発疑い」等があれば、T0判定(腫瘍を認めない) の可能性も忘れず検討。


【最終チェック(強制ルール)】
pure GGO が T1c 以上になっていないか 再確認
「腫瘍なし」と明記があれば T0 に矛盾していないか
胸壁・大血管浸潤などのキーワードを再スキャンして T4 を見落としていないか
副腫瘍結節 位置の再チェックで T3/T4/M1a を付け忘れていないか
充実成分径が0mmの純粋GGOがT1a以上になっていないか
充実成分径≦5mmかつ全体径≦30mmのmixed GGOがT1miとして考慮されているか

肺癌のTNM分類について、以下のフォーマットに従って出力してください。
JSON形式で T（原発腫瘍）、N（リンパ節転移）、M（遠隔転移）に関する情報をそれぞれ記載してください。
各値は明確な記載がない場合は最も近いと思われるものとしてください（空欄にならないように各カテゴリは1つの候補に絞ってください）。

{
  "T": {
    "stage": "候補の中から選択",
    "details": "原発腫瘍のサイズや特徴の記載"
  },
  "N": {
    "stage": "候補の中から選択",
    "details": "リンパ節転移の有無や部位の記載"
  },
  "M": {
    "stage": "候補の中から選択",
    "details": "遠隔転移の有無や転移先の記載"
  }
}

【答えの候補】

T（原発腫瘍）:Tis, T1mi, T1a, T1b, T1c, T2a, T2b, T3, T4
N（リンパ節転移）:NX, N0, N1, N2, N3
M（遠隔転移）: M0, M1a, M1b, M1c

【読影レポート】\n
"""

def generate_prediction(report, prompt=prompt, model=model):
    full_text = prompt + report
    # generation_config 辞書を作成する
    generation_config = genai.GenerationConfig(temperature=0,
                                            top_p=1.0,
                                            top_k=0,
                                           )
    response = model.generate_content(
        full_text,
        generation_config=generation_config # generation_config を渡す
    )
    return response.text.strip()


# ==============================================
# 7) 実行例
# ==============================================
print("== 1件目のレポートを取得して推論する例 ==")
report_id = valid_df.loc[0, "id"]
report_text = get_report(report_id)

print("-- 入力レポート --")
print(report_text[:300] + " ... (省略)")

generated = generate_prediction(report_text)
print("\n-- モデル生成結果 --")
print(generated)

parsed_dict = extract_json(generated)
print("\n-- JSONにパースした結果 --")
print(parsed_dict)


== 1件目のレポートを取得して推論する例 ==
-- 入力レポート --
左肺門部に 37mm 大の腫瘤影を認め、ご指摘の肺癌が疑われます。
縦隔に有意なリンパ節腫大は認めません。
胸水はありません。
背部皮下に腫瘤を認め、粉瘤などと思われます。 ... (省略)

-- モデル生成結果 --
Here's a breakdown of the thought process to arrive at the JSON output:

1. **Understand the Goal:** The primary goal is to classify the lung cancer described in the radiology report according to the TNM staging system (8th edition, 2017) and present the findings in a structured JSON format.

2. **Initial Scan of the Report:**  Read the report to get a general understanding. Key phrases are "左肺門部に 37mm 大の腫瘤影" and "肺癌が疑われます." This immediately tells us there's a tumor and it's likely malignant. The size (37mm) is a crucial piece of information for T-staging.

3. **T-Staging - The Core Task:**  The T-stage is determined by the primary tumor's characteristics. The instructions provide a detailed breakdown.

    * **Step 1: Tumor Presence:** The report clearly states a tumor is present ("腫瘤影を認め"). This rules out T0. There's no mention of cytolo

全データに対して推論を実行して、サブミッションのための csv を作っていきます。

In [ ]:
from google.colab import files

from tqdm import tqdm
import pandas as pd
import os
import time
import requests

# google-generativeai パッケージを最新バージョンにアップグレード
!pip install -q -U google-generativeai --upgrade

import google.generativeai as genai # genaiを再インポート

# valid_df, get_report, generate_prediction, extract_json が定義済みとして進めます。

# 全データを辞書形式に変換
rows = valid_df.to_dict(orient="records")

# 出力を格納するためのリスト
results = []

# すべてのレポートに対して推論を実行
for row in tqdm(rows, desc="Processing rows"):
    report_id = row["id"]
    report_text = get_report(report_id)            # レポート本文を読み込み

    # 再試行メカニズムを追加
    max_retries = 3  # 最大再試行回数
    retries = 0
    while retries < max_retries:
        try:
            generated_text = generate_prediction(report_text)  # Google Generative AI で推論
            break  # 成功したらループを抜ける
        except (ConnectionError, requests.exceptions.RequestException, genai.errors.TooManyRequestsError) as e:
            # TooManyRequestsError を追加
            print(f"Error occurred: {e}")
            if isinstance(e, genai.errors.TooManyRequestsError):
                print("Too many requests. Waiting for 60 seconds before retrying...")
                time.sleep(60)  # TooManyRequests の場合は60秒待機
            else:
                retries += 1
                time.sleep(5)  # 再試行前に少し待つ
    else:
        print(f"Failed to generate prediction for report ID: {report_id} after {max_retries} retries.")
        continue  # 最大再試行回数を超えた場合はスキップ

    extracted = extract_json(generated_text)       # JSON を抽出
    results.append(extracted)

    # リクエスト間隔を空ける (例: 5秒)
    time.sleep(5)  # 間隔を長くする

# JSON から T, N, M を取り出して Pandas DataFrame へ変換
tnm_data = [
    {
        "t": entry["T"]["stage"],
        "n": entry["N"]["stage"],
        "m": entry["M"]["stage"]
    }
    for entry in results
]

results_df = valid_df.copy()
results_df[["t", "n", "m"]] = pd.DataFrame(tnm_data, index=results_df.index)

# 確認用に先頭数行を表示
display(results_df.head(3))

# CSVに書き出し (ファイル名は任意でOK)
results_df.to_csv("submission.csv", index=False)



Processing rows: 100%|██████████| 54/54 [10:11<00:00, 11.32s/it]


,id,t,n,m
0,147290,T2a,N0,M0
1,241752,Tis,N0,M0
2,876951,T2a,N0,M0


# 結果

プロンプトの修正で15%程度精度向上

**improvement**
```
{"Joint accuracy (fine)": 0.8333333333333334,
"T accuracy (fine)": 0.9074074074074074,
"N accuracy (fine)": 0.9259259259259259,
"M accuracy (fine)": 0.9814814814814815,
"Joint accuracy (coarse)": 0.8518518518518519,
"T accuracy (coarse)": 0.9259259259259259,
"N accuracy (coarse)": 0.9259259259259259,
"M accuracy (coarse)": 0.9814814814814815}
```





**models/gemini-2.0-flash-thinking-exp-1219**
```
{"Joint accuracy (fine)": 0.6851851851851852,
 "T accuracy (fine)": 0.7777777777777778,
 "N accuracy (fine)": 0.9444444444444444,
 "M accuracy (fine)": 0.9259259259259259,
 "Joint accuracy (coarse)": 0.7777777777777778,
 "T accuracy (coarse)": 0.8888888888888888,
 "N accuracy (coarse)": 0.9444444444444444,
 "M accuracy (coarse)": 0.9259259259259259}
```
